In [2]:
from __future__ import print_function

import math

from IPython import display
from matplotlib import cm
from matplotlib import gridspec
import matplotlib.pyplot as plt

import tensorflow as tf
import numpy as np
import pandas as pd
import sqlalchemy as sa

from keras.utils import to_categorical

import pyproj
import json

from functools import partial
from shapely.geometry import shape, Point
from shapely.ops import transform
from itertools import product 

import sklearn.metrics as metrics
from tensorflow.python.data import Dataset

In [5]:
movies_dataframe =  pd.read_csv("/Users/larasikanadela/Documents/data_science/movie_metadata.csv", sep=",")
movies_dataframe

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0
5,Color,Andrew Stanton,462.0,132.0,475.0,530.0,Samantha Morton,640.0,73058679.0,Action|Adventure|Sci-Fi,...,738.0,English,USA,PG-13,263700000.0,2012.0,632.0,6.6,2.35,24000
6,Color,Sam Raimi,392.0,156.0,0.0,4000.0,James Franco,24000.0,336530303.0,Action|Adventure|Romance,...,1902.0,English,USA,PG-13,258000000.0,2007.0,11000.0,6.2,2.35,0
7,Color,Nathan Greno,324.0,100.0,15.0,284.0,Donna Murphy,799.0,200807262.0,Adventure|Animation|Comedy|Family|Fantasy|Musi...,...,387.0,English,USA,PG,260000000.0,2010.0,553.0,7.8,1.85,29000
8,Color,Joss Whedon,635.0,141.0,0.0,19000.0,Robert Downey Jr.,26000.0,458991599.0,Action|Adventure|Sci-Fi,...,1117.0,English,USA,PG-13,250000000.0,2015.0,21000.0,7.5,2.35,118000
9,Color,David Yates,375.0,153.0,282.0,10000.0,Daniel Radcliffe,25000.0,301956980.0,Adventure|Family|Fantasy|Mystery,...,973.0,English,UK,PG,250000000.0,2009.0,11000.0,7.5,2.35,10000


In [7]:
actors_dataframe = pd.read_csv("/Users/larasikanadela/Documents/data_science/actor-metrics.csv", sep=",")
actors_dataframe

,year,film,actor,characters,imdb,gender,race,race_simple,words,sentences
0,1989,born-on-the-fourth-of-july,Dale Dye,COLONEL,http://www.imdb.com/name/nm0245653/,male,White/Caucasian,White,216.0,15.0
1,1989,born-on-the-fourth-of-july,David Warshofsky,LIEUTENANT,http://www.imdb.com/name/nm0913175/,male,White/Caucasian,White,290.0,40.0
2,1989,born-on-the-fourth-of-july,Frank Whaley,TIMMY,http://www.imdb.com/name/nm0001844/,male,White/Caucasian,White,104.0,14.0
3,1989,born-on-the-fourth-of-july,Jerry Levine,STEVE,http://www.imdb.com/name/nm0505842/,male,White/Caucasian,White,300.0,35.0
4,1989,born-on-the-fourth-of-july,John F. Kennedy,KENNEDY,NaN,male,White/Caucasian,White,217.0,5.0
5,1989,born-on-the-fourth-of-july,Josh Evans,TOMMY,http://www.imdb.com/name/nm0262923/,male,White/Caucasian,White,140.0,22.0
6,1989,born-on-the-fourth-of-july,Kyra Sedgwick,DONNA,http://www.imdb.com/name/nm0001718/,female,White/Caucasian,White,188.0,30.0
7,1989,born-on-the-fourth-of-july,Lane R. Davis,STEVIE,http://www.imdb.com/name/nm0205001/,male,White/Caucasian,White,366.0,34.0
8,1989,born-on-the-fourth-of-july,Markus Flanagan,DOCTOR,http://www.imdb.com/name/nm0281074/,male,White/Caucasian,White,161.0,16.0
9,1989,born-on-the-fourth-of-july,Norma Moore,MOM,http://www.imdb.com/name/nm0601689/,female,White/Caucasian,White,348.0,45.0


In [8]:
biopics_dataframe = pd.read_csv("/Users/larasikanadela/Documents/data_science/biopics.csv", sep=",")
biopics_dataframe

,title,site,country,year_release,box_office,director,number_of_subjects,subject,type_of_subject,race_known,subject_race,person_of_color,subject_sex,lead_actor_actress
0,10 Rillington Place,http://www.imdb.com/title/tt0066730/,UK,1971,-,Richard Fleischer,1,John Christie,Criminal,Unknown,NaN,0,Male,Richard Attenborough
1,12 Years a Slave,http://www.imdb.com/title/tt2024544/,US/UK,2013,$56.7M,Steve McQueen,1,Solomon Northup,Other,Known,African American,1,Male,Chiwetel Ejiofor
2,127 Hours,http://www.imdb.com/title/tt1542344/,US/UK,2010,$18.3M,Danny Boyle,1,Aron Ralston,Athlete,Unknown,NaN,0,Male,James Franco
3,1987,http://www.imdb.com/title/tt2833074/,Canada,2014,-,Ricardo Trogi,1,Ricardo Trogi,Other,Known,White,0,Male,Jean-Carl Boucher
4,20 Dates,http://www.imdb.com/title/tt0138987/,US,1998,$537K,Myles Berkowitz,1,Myles Berkowitz,Other,Unknown,NaN,0,Male,Myles Berkowitz
5,21,http://www.imdb.com/title/tt0478087/,US,2008,$81.2M,Robert Luketic,1,Jeff Ma,Other,Known,Asian American,1,Male,Jim Sturgess
6,24 Hour Party People,http://www.imdb.com/title/tt0274309/,UK,2002,$1.13M,Michael Winterbottom,1,Tony Wilson,Musician,Known,White,0,Male,Steve Coogan
7,42,http://www.imdb.com/title/tt0453562/,US,2013,$95M,Brian Helgeland,1,Jackie Robinson,Athlete,Known,African American,1,Male,Chadwick Boseman
8,8 Seconds,http://www.imdb.com/title/tt0109021/,US,1994,$19.6M,John G. Avildsen,1,Lane Frost,Athlete,Unknown,NaN,0,Male,Luke Perry
9,84 Charing Cross Road,http://www.imdb.com/title/tt0090570/,US/UK,1987,$1.08M,David Hugh Jones,2,Frank Doel,Author,Unknown,NaN,0,Male,Anthony Hopkins
